You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [5]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [6]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [7]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [8]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [9]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [10]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [11]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [12]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 

Title: The Future of Artificial Intelligence: Trends, Players, and News

Outline:

I. Introduction
- Brief overview of Artificial Intelligence (AI) and its impact on various industries
- Importance of staying updated on the latest trends and news in AI

II. Latest Trends in Artificial Intelligence
- Advancements in machine learning algorithms
- Rise of AI-powered chatbots and virtual assistants
- Integration of AI in healthcare, finance, and transportation industries

III. Key Players in the 

I now can give a great answer

Final Answer:
# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction
Artificial Intelligence (AI) has become a transformative force across various industries, revolutionizing how businesses operate and individuals interact with technology. From healthcare to finance, AI has the potential to streamline processes, enhance decision-making, and drive innovation. Staying updated on the latest trends and news in AI is crucial for individuals and organizations looking to leverage the power of this cutting-edge technology effectively.

## Latest Trends in Artificial Intelligence
One of the most significant trends in AI is the continuous advancements in machine learning algorithms. These algorithms are becoming increasingly sophisticated, enabling AI systems to learn from vast amounts of data and improve their performance over time. Additionally, the rise of AI-powered chatbots and virtual assistants has transformed customer service an

- Display the results of your execution as markdown in the notebook.

In [13]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction
Artificial Intelligence (AI) has become a transformative force across various industries, revolutionizing how businesses operate and individuals interact with technology. From healthcare to finance, AI has the potential to streamline processes, enhance decision-making, and drive innovation. Staying updated on the latest trends and news in AI is crucial for individuals and organizations looking to leverage the power of this cutting-edge technology effectively.

## Latest Trends in Artificial Intelligence
One of the most significant trends in AI is the continuous advancements in machine learning algorithms. These algorithms are becoming increasingly sophisticated, enabling AI systems to learn from vast amounts of data and improve their performance over time. Additionally, the rise of AI-powered chatbots and virtual assistants has transformed customer service and communication channels, providing personalized experiences to users. Furthermore, the integration of AI in sectors such as healthcare, finance, and transportation is reshaping industries by optimizing operations, enhancing decision-making, and improving overall efficiency.

## Key Players in the AI Industry
Several key players are leading the way in the AI industry, driving innovation and pushing the boundaries of what is possible with artificial intelligence. Companies like Google's DeepMind, IBM Watson, Tesla's Autopilot, Amazon's Alexa, and Microsoft's Cognitive Services are at the forefront of AI development, creating groundbreaking solutions that are shaping the future of technology. These companies are investing heavily in research and development to push the boundaries of AI capabilities and unlock new possibilities for businesses and consumers alike.

## Noteworthy News in Artificial Intelligence
Recent breakthroughs in natural language processing have enabled AI systems to understand and generate human language more accurately than ever before. However, ethical concerns surrounding AI development, such as bias in algorithms and data privacy issues, continue to be topics of debate and discussion. The impact of AI on job markets and the workforce is also a significant consideration, as automation and AI technologies continue to reshape the way we work and the skills required in the future.

## Target Audience Analysis
The target audience for information on Artificial Intelligence includes tech-savvy individuals interested in cutting-edge technologies, business professionals seeking to leverage AI for their companies, and students and researchers in the field of artificial intelligence. These audiences can benefit from staying informed about the latest trends, key players, and news in the AI industry to make informed decisions and stay ahead of the curve in their respective fields.

In conclusion, Artificial Intelligence is a rapidly evolving field with significant implications for various industries and society as a whole. By staying informed about the latest trends and news in AI, individuals and organizations can harness the power of this transformative technology to drive innovation and growth. Embracing the opportunities that AI presents can lead to new possibilities and advancements in personal and professional endeavors.

**Call to Action:**
Stay informed about the latest trends and news in Artificial Intelligence by following reputable sources and industry leaders. Embrace the opportunities that AI presents for innovation and growth in your personal and professional endeavors. 

*Resources:*
- Forbes article on AI trends in 2021
- Gartner's report on key players in the AI industry
- Harvard Business Review on ethical considerations in AI development

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [14]:
topic = "sexuality"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on sexuality.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: Exploring Sexuality: Understanding Key Trends and Empowering Choices

Outline:
I. Introduction
- Brief overview of the importance of understanding sexuality
- Statement of purpose for the blog article

II. Latest Trends in Sexuality
A. Gender Identity and LGBTQ+ Rights
B. Sexual Health and Education
C. Consent and Boundaries
D. Online Dating and Relationships

III. Key Players in the Field of Sexuality
A. Advocacy Organizations
B. Researchers and Educators
C. Influencers and Thought Leaders
D. Government Agencies



I now can give a great answer

Final Answer: 

# Exploring Sexuality: Understanding Key Trends and Empowering Choices

## Introduction
Sexuality is a fundamental aspect of human identity and plays a crucial role in shaping individuals' lives. Understanding one's sexuality is essential for personal growth and empowerment. In this blog post, we will delve into the latest trends, key players, noteworthy news, target audience analysis, and SEO keywords related to sexuality to provide a comprehensive guide for readers.

## Latest Trends in Sexuality

### Gender Identity and LGBTQ+ Rights
The recognition and acceptance of diverse gender identities and sexual orientations have been significant trends in recent years. The LGBTQ+ community has made strides in advocating for equal rights and representation in various sectors of society.

### Sexual Health and Education
Promoting sexual health education and awareness is crucial in ensuring individuals make informed decisions about their sexual we

In [15]:
Markdown(result)

# Exploring Sexuality: Understanding Key Trends and Empowering Choices

## Introduction
Sexuality is a fundamental aspect of human identity and plays a crucial role in shaping individuals' lives. Understanding one's sexuality is essential for personal growth and empowerment. In this blog post, we will delve into the latest trends, key players, noteworthy news, target audience analysis, and SEO keywords related to sexuality to provide a comprehensive guide for readers.

## Latest Trends in Sexuality

### Gender Identity and LGBTQ+ Rights
The recognition and acceptance of diverse gender identities and sexual orientations have been significant trends in recent years. The LGBTQ+ community has made strides in advocating for equal rights and representation in various sectors of society.

### Sexual Health and Education
Promoting sexual health education and awareness is crucial in ensuring individuals make informed decisions about their sexual well-being. Access to comprehensive sexual education can help reduce stigma and improve overall health outcomes.

### Consent and Boundaries
The importance of consent and respecting boundaries in sexual interactions has gained prominence in discussions about healthy relationships. Understanding and practicing consent is essential for creating safe and respectful environments.

### Online Dating and Relationships
The rise of online dating platforms has transformed the way people connect and form relationships. Navigating the digital landscape requires awareness of online safety practices and communication skills.

## Key Players in the Field of Sexuality

### Advocacy Organizations
Organizations such as Planned Parenthood and The Trevor Project play a vital role in advocating for sexual health, LGBTQ+ rights, and comprehensive education. Their work contributes to creating a more inclusive and informed society.

### Researchers and Educators
Researchers and educators in the field of sexuality provide valuable insights and resources for understanding human sexuality. Their work informs policies, programs, and interventions aimed at promoting sexual well-being.

### Influencers and Thought Leaders
Influencers and thought leaders contribute to shaping public discourse on sexuality through their platforms and advocacy efforts. Their influence helps raise awareness and promote positive attitudes towards diverse sexual identities.

### Government Agencies
Government agencies play a significant role in developing and implementing policies related to sexuality, including healthcare services, education programs, and anti-discrimination laws. Their involvement is crucial in advancing the rights and well-being of all individuals.

## Noteworthy News in the Realm of Sexuality

### Legislative Updates
Recent legislative updates related to sexuality have addressed issues such as LGBTQ+ rights, reproductive health, and sexual violence prevention. These changes reflect evolving societal norms and priorities.

### Research Findings
New research findings in the field of sexuality provide valuable insights into human behavior, attitudes, and practices. Stay informed about the latest research to enhance your understanding of sexual health and relationships.

### Social Movements
Social movements advocating for sexual rights, gender equality, and LGBTQ+ visibility continue to make an impact on society. Get involved in these movements to support positive change and empowerment for all individuals.

### Media Representation
Media representation of diverse sexual identities and relationships plays a crucial role in shaping public perceptions and attitudes. Support media that portrays authentic and inclusive portrayals of sexuality.

## Target Audience Analysis
This blog post is intended for young adults aged 18-35, individuals exploring their own sexuality, parents seeking resources to discuss sexuality with their children, and educators and healthcare professionals. By addressing a diverse audience, we aim to provide valuable insights and resources for all individuals interested in learning more about sexuality.

## SEO Keywords
- Sexuality trends
- Gender identity
- Sexual health education
- LGBTQ+ rights
- Consent and boundaries
- Online dating
- Relationships

## Resources
- Planned Parenthood
- The Trevor Project
- The Kinsey Institute
- American Sexual Health Association

## Call to Action
We encourage readers to continue educating themselves on sexuality by exploring the resources provided in this blog post. Promote open and respectful dialogue on the topic to create a more inclusive and informed community. Together, we can empower individuals to make informed choices and embrace their sexuality with confidence.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).